In [13]:
from meteofrance_api import MeteoFranceClient
from colorama import Fore, Style, init
from datetime import datetime, timezone
import time

init(autoreset=True)

LATITUDE = 43.6045
LONGITUDE = 1.4440


def convert_to_local_dt(dt):
    """Renvoie un datetime local à partir d'un timestamp (int) ou ISO (str)."""
    if isinstance(dt, int):
        dt_utc = datetime.fromtimestamp(dt, tz=timezone.utc)
    elif isinstance(dt, str):
        dt_utc = datetime.fromisoformat(dt.replace("Z", "+00:00"))
    else:
        raise ValueError("Format de date non reconnu pour dt.")

    return dt_utc.astimezone() 


def format_hour(dt_local):
    return dt_local.strftime("%H:%M")


try:
    client = MeteoFranceClient()
    now = datetime.now().astimezone()

    rain_forecast = client.get_rain(latitude=LATITUDE, longitude=LONGITUDE)
    print(Fore.CYAN + f"\n🌧️  Prévisions de pluie (1h à venir – heures futures uniquement) :\n")

    for f in rain_forecast.forecast:
        dt_local = convert_to_local_dt(f["dt"])
        if dt_local >= now: 
            print(f"{Fore.YELLOW}[{format_hour(dt_local)}] "
                  f"{Fore.WHITE}{f['desc']}  (intensité : {f['rain']})")

    print("\n" + Fore.MAGENTA + "-"*65 + "\n")
    forecast = client.get_forecast(latitude=LATITUDE, longitude=LONGITUDE)
    hourly_forecast = forecast.forecast

    print(Fore.CYAN + "🌡️  Prévisions horaires (seulement les heures futures) :\n")

    for h in hourly_forecast:
        dt_local = convert_to_local_dt(h["dt"])
        if dt_local < now:
            continue 

        heure = format_hour(dt_local)
        desc = h["weather"]["desc"] if "weather" in h else "Non précisé"

        temp_dict = h.get("T", {})
        temp_val = temp_dict.get("value", "N/A")
        ressentie = temp_dict.get("windchill", "N/A")
        vent = h.get("wind", {}).get("speed", "N/A")

        if "clair" in desc.lower(): icon = "☀️"
        elif "nuage" in desc.lower(): icon = "☁️"
        elif "pluie" in desc.lower(): icon = "🌧️"
        elif "éclair" in desc.lower() or "orage" in desc.lower(): icon = "⛈️"
        elif "neige" in desc.lower(): icon = "❄️"
        else: icon = "🌤️"

        if temp_val != "N/A":
            if temp_val < 5: color = Fore.CYAN
            elif temp_val < 15: color = Fore.BLUE
            elif temp_val < 25: color = Fore.YELLOW
            else: color = Fore.RED
        else:
            color = Fore.WHITE

        print(f"{Fore.GREEN}{heure} | {icon} {desc:<15} | "
              f"{color}🌡️ {temp_val}°C (ressentie {ressentie}°C) | "
              f"{Fore.LIGHTMAGENTA_EX}💨 {vent} km/h")

except Exception as e:
    print(f"❌ Une erreur s'est produite : {e}")
    print("Assurez-vous que vos coordonnées sont valides et que l'API fonctionne.")
    



🌧️  Prévisions de pluie (1h à venir – heures futures uniquement) :

[15:25] Temps sec  (intensité : 1)
[15:30] Temps sec  (intensité : 1)
[15:35] Temps sec  (intensité : 1)
[15:40] Temps sec  (intensité : 1)
[15:45] Temps sec  (intensité : 1)
[15:50] Temps sec  (intensité : 1)
[16:00] Temps sec  (intensité : 1)
[16:10] Temps sec  (intensité : 1)
[16:20] Temps sec  (intensité : 1)

-----------------------------------------------------------------

🌡️  Prévisions horaires (seulement les heures futures) :

16:00 | 🌤️ Ciel voilé      | 🌡️ 19.4°C (ressentie 22.5°C) | 💨 10 km/h
17:00 | 🌤️ Ciel voilé      | 🌡️ 18.7°C (ressentie 22.1°C) | 💨 11 km/h
18:00 | 🌤️ Ciel voilé      | 🌡️ 18.4°C (ressentie 21.7°C) | 💨 11 km/h
19:00 | ☁️ Très nuageux    | 🌡️ 18.1°C (ressentie 21.6°C) | 💨 11 km/h
20:00 | 🌤️ Couvert         | 🌡️ 17.7°C (ressentie 21.2°C) | 💨 11 km/h
21:00 | 🌤️ Ciel voilé      | 🌡️ 17.9°C (ressentie 21.6°C) | 💨 11 km/h
22:00 | 🌤️ Ciel voilé      | 🌡️ 18°C (ressentie 21.7°C) | 💨 11 km/h
23